# Overview:
This Notebook goes through the following processing steps:
1. Locate potential crossing locations between all beams
2. Identifies valid crossovers from the list generated in step 1 according to user-specified criteria, and estimates elevation change (dh) at these crossing locations
3. Temporally subsets results

# Set up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def read_h5(fname, vnames=[]):
    """ Simple HDF5 reader. """ #from IS2 hackweek tutorial: https://github.com/ICESAT-2HackWeek/2020_ICESat-2_Hackweek_Tutorials/blob/main/08.HDF5_and_ICESat-2_data_files/intro-is2-files_rendered.ipynb
    import h5py
    with h5py.File(fname, 'r') as f:
        return [f[v][:] for v in vnames]

# Step 1: Locate Potential Crossing Locations
(note: This function can take a long time (up to an hour or so) depending on your machine)

In [ ]:
def get_coarse_xovers(data_dir, outfile):
    """
    performs pairwise search for intersection between all tracks in the list. For every pair of tracks, this algorithm:

    -Divides the tracks into 10-km segements (a lengthscale over which we assume the tracks can be approximated as linear)
    -Fits a line to each the segmenets using linear regression
    -Solves for the intersection between the two segements
    -If the intersection point lies within the lat/lon bounds for the given segments, this is flagged as a valid crossover, and the files names, dates, and coordinates of the intersection are recorded and saves the results in an hdf5 file listing
    the file name, rgt, dates, and intersection coordinates in UTM (zone 6) for each crossover are recorded
    
    The list of potential crossovers is saved as an hdf5 file. 
    
    Parameters:
    data_dir: directory with ATL06 files in reduced hdf5 format
    outfile: name of output hdf5 file
    """
    #import necessary packages
    import time
    import os
    import h5py
    import numpy as np
    import scipy.stats as stats
    import pandas as pd
    import glob as glob
    from pyproj import Transformer
    
    start = time.time() #start timer
    
    #define some helper functions
    def read_h5(fname, vnames=[]):
        """ Simple HDF5 reader. """
        with h5py.File(fname, 'r') as f:
            return [f[v][:] for v in vnames]
        
    def get_slope_intercept(x, y):
        """
        performs a linear interpolation and returns the slope and intercept
        
        """
        m_a, c_a, r_a, p_a, sigma_a = stats.linregress(x,y)
        return m_a, c_a
    def get_rgt(fname):
        return int(fname[31:35])
    def get_beam(fname):
        return(fname[47:51])
    def get_date(fname):
        return fname[16:24]
    #gather files
    os.chdir(data_dir)
    files = glob.glob('*.h5')
    #initialize lists
    track_1_name = [] #file name for first track (string)
    track_2_name = []
    date_1 = [] #date of first track (int)
    date_2 = []
    intersection_x = [] #x-coordinate of estimated intersection (in UTM)
    intersection_y = [] #x-coordinate of estimated intersection (in UTM)
    #define latitudinal bands of 10 km to search (assuming tracks are ~linear on 10-km lengthscale)
    lat_bands = np.linspace(7600000, 7730000, 14)
    #loop through files
    for i in range(0, len(files)):
        track_name_a = files[i] #get filename
        lon_a, lat_a, date_a =  read_h5(track_name_a, ['lon', 'lat', 't_year'])
        #filter out points where the lon/lat is 0
        lon_a = lon_a[(lon_a != 0) & (lat_a != 0)]
        lat_a = lat_a[(lon_a != 0) & (lat_a != 0)]
        #project into UTM Zone 6
        transformer = Transformer.from_crs("epsg:4326", "epsg:32606")
        x_a, y_a = transformer.transform(lat_a, lon_a)
        for j in range(i+1, len(files)): #get second track for comparison
            if (j != i):
                track_name_b = files[j] #get filename
                lon_b, lat_b, date_b =  read_h5(track_name_b, ['lon', 'lat', 't_year'])
                lon_b = lon_b[(lon_b != 0) & (lat_b != 0)]
                lat_b = lat_b[(lon_b != 0) & (lat_b != 0)]
                x_b, y_b = transformer.transform(lat_b, lon_b)
                #if the two tracks are from the same rgt, check to see if they were direct repeats:
                if((get_rgt(track_name_a) == get_rgt(track_name_b)) &(get_date(track_name_a) != get_date(track_name_b)) & \
                   (get_beam(track_name_a) == get_beam(track_name_b))):
                    distance = []
                    for a in range(0, len(lon_a)):
                        #get distance between each point in track a and the closest point in track b
                        x1 = x_a[a]
                        y1 = y_a[a]
                        distance.append(np.min(np.sqrt((x1 - x_b)**2 + (y1-y_b)**2))) 
                    if np.mean(distance) < 30: #if tracks are within 30m of each other, assume it's a repeat and move to next iteration (this may be a higher threshold than needed)
                        continue
                #loop through the latitudinal bands to find crossover
                for l in range(0, (len(lat_bands) -1)):
                    #define latitudinal band
                    lat_min = lat_bands[l]
                    lat_max = lat_bands[l+1]
                    #subset track coordinates to latitudinal band:
                    x_a_l = x_a[(y_a <= lat_max) & (y_a >= lat_min)]
                    y_a_l = y_a[(y_a <= lat_max) & (y_a >= lat_min)]
                    x_b_l = x_b[(y_b <= lat_max) & (y_b >= lat_min)]
                    y_b_l = y_b[(y_b <= lat_max) & (y_b >= lat_min)]
                    if ((len(x_a_l)<2) or (len(x_b_l) <2)):
                        continue #move to next band if there's not enough data
                    
                    #perform linear fit on data to get the slope (m) and intercept(c)
                    m_a, c_a = get_slope_intercept(x_a_l, y_a_l)
                    m_b, c_b = get_slope_intercept(x_b_l, y_b_l)
                    
                    #find min/max longitude values in this band:
                    lon_full = np.concatenate((x_a_l, x_b_l))
                    lon_max = np.max(lon_full)
                    lon_min = np.min(lon_full)
                    lat_full = np.concatenate((y_a_l, y_b_l))
                    lat_max = np.max(lat_full)
                    lat_min = np.min(lat_full)

                    if (m_a != m_b): #only solve for non-parallel lines
                        #solve for crossing point between the two tracks
                        x_i = (c_b - c_a)/(m_a - m_b)
                        y_i = (c_a*m_b - c_b*m_a)/(m_b - m_a) 
                        #check if solution is within bounds of 10 km band
                        if ((x_i < (lon_max)) & (x_i >(lon_min)) &(y_i < (lat_max)) & (y_i > (lat_min))):
                            #order lists such that earlier date comes first
                            if (date_a[0] > date_b[0]):
                                #save data to lists
                                track_1_name.append(track_name_b)
                                track_2_name.append(track_name_a)
                                date_1.append(date_b[0])
                                date_2.append(date_a[0])
                                intersection_x.append(x_i)
                                intersection_y.append(y_i)
                            elif(date_b[0] > date_a[0]):
                                track_1_name.append(track_name_a)
                                track_2_name.append(track_name_b)
                                date_1.append(date_a[0])
                                date_2.append(date_b[0])
                                intersection_x.append(x_i)
                                intersection_y.append(y_i)
                            break #stop looping through bands if crossover found
        #progress check at every 50th track:
        if (i%50 ==0):
            print('progress: files checked = ', i)
    #convert data into dictionary, then dataframe
    os.chdir('..')
    data_dict = {'File_1': track_1_name, 'File_2': track_2_name, \
                'date_1': date_1, 'date_2': date_2, 'x': intersection_x, 'y': intersection_y}
    data = pd.DataFrame(data_dict)
    data.to_hdf(outfile, key = 'df') #save file
    end = time.time()
    print('time elapsed: ', (end - start)) #track time to complete

In [ ]:
get_coarse_xovers('Anaktuvuk_v2_ATL06_reduced_all_slopes', 'course_xovers_all_slopes_new_demo.h5')

# Step 2: Identify valid crossovers and estimate elevation change (and other statistics)

In [ ]:
def crossovers(course_xovers, data_dir, radius, n_min, x_bounds, y_bounds, outfile, n_max = None, slope_filter = None):
    """"
    takes an h5 file with possible crossovers, generated using get_course_xovers, and calculates crossovers and relevant 
    statistics for all points that have at least n_min points within the specified radius of the crossing location. First, 
    the crossing location is re-estimated using a more localized linear interpolation based on the specified radius. The height 
    of each track at the crossing location is estimated using a linear interpolation of all points within the specified radius 
    from the crossing location, unless n_max is specified. The data can be further subsetted by a bounding box and along-track 
    slope threshold. The data is saved to the specified outfile in csv format.
    
    Parameters:
    
    course_xovers(string): file with list of potential crossovers, generated via get_course_xovers
    
    data_dir(string): directory where ATL06 (in reduced h5 format) file are located
    
    radius: radius over which to search for data points and perform linear interpolation
    
    n_min (int): minimum number of points required within interpolation radius to be counted as a valid crossover
    
    x_bounds(array): x coordinates of bounding box to search over, as [lon_min, lon_max] in epsg 4326
    
    y_bounds(array): y coordinates of bounding box to search over, as [lat_min, lat_max] in epsg 4326
    
    outfile: name of output file (should be a csv)
    
    n_max (int): If specified, only use the n_max closest points when performing the linear interpolation
    
    slope_filter(double, m/m): If specified, ATL06 segments with an along-track slope higher than the filter will be removed 
    
    """
    import h5py
    import numpy as np
    import scipy.stats as stats
    from scipy import interpolate
    import pandas as pd
    from pyproj import Transformer
    import time
    import os
    #define function for reading in hdf5 files
    def read_h5(fname, vnames=[]):
        """ Simple HDF5 reader. """
        with h5py.File(fname, 'r') as f:
            return [f[v][:] for v in vnames]
    #define functions for pulling relative attributes out of file names
    def get_rgt(fname):
        return int(fname[31:35])
    def get_cycle(fname):
        return int(fname[36])
    def get_date(fname):
        return fname[16:24]
    def get_beam(fname):
        return(fname[47:51])
        
    
    
    def intersection(x1, y1, x2, y2):
        """
        uses a linear interpolation to estimate the intersection of two data sets
        """
        #perform linear fit to each dataset
        m_a, c_a, r_a, p_a, sigma_a = stats.linregress(x1,y1)
        m_b, c_b, r_b, p_b, sigma_b = stats.linregress(x2,y2)
        x_full = np.concatenate((x1, x2))
        y_full = np.concatenate((y1, y2))
        x_max = np.max(x_full)
        x_min = np.min(x_full)
        y_max = np.max(y_full)
        y_min = np.min(y_full)
        if (m_a != m_b): #only solve for non-parallel lines using slope (m) and intercept (c)
            lon_i = (c_b - c_a)/(m_a - m_b)
            lat_i = (c_a*m_b - c_b*m_a)/(m_b - m_a)
            if ((lon_i < (x_max)) & (lon_i >(x_min)) &(lat_i < (y_max)) & (lat_i > (y_min))): #make sure solution is in lat/lon range of the two tracks
                return lon_i, lat_i
            else:
                lon_i = float("nan")
                lat_i = float("nan")
                return lon_i, lat_i
        else:
            lon_i = float("nan")
            lat_i = float("nan")
            return lon_i, lat_i
    start = time.time() #start timer
    transformer = Transformer.from_crs("epsg:4326", "epsg:32606") #transform from ESPG 4326 (lat/lon) to UTM zone 6
    transformer_i = Transformer.from_crs("epsg:32606", "epsg:4326") #transform from UTM zone 6 to ESPG 4326 (lat/lon)
    
    #initialize lists for storage
    #crossover coordinates in UTM
    crossover_x = [] 
    crossover_y = []
    #estimated height at crossing point for each track (m)
    height_1 = [] 
    height_2 = [] 
    dh_elev = [] #estimated elevation change at crossing point (m)
    #propogated uncertainty of crossing height on each track (m)
    sigma_1 = [] 
    sigma_2 = [] 
    sigma_c = [] #propogated uncertainty of elevation change (m)
    #time/date of each track
    time_1 = [] 
    time_2 = []
    delta_time = [] #time interval between the two tracks
    #file name of each track
    f1 = [] 
    f2 = [] 
    #number of points used for interpolation for each track
    n1 = [] 
    n2 = []
    #mean number of photons per segement for each track
    n_photons_1 = []
    n_photons_2 = []
    #mean along-track slope for each track 
    dh_dx_1 = []
    dh_dx_2 = []
    #mean across-track slope for each track
    dh_dy_1 = []
    dh_dy_2 = []
    #sum of the square of the residuals from the linear fit for each track
    resid_1 = []
    resid_2 = []
    
    #read in crossover list
    course_crossovers = pd.read_hdf(course_xovers)
    os.chdir(data_dir)
    for f in range(0, len(course_crossovers)):
        #read in files and reproject to UTM
        file_1 = (course_crossovers['File_1']).values[f]
        lon1, lat1, t1, h1, dh_dx1, dh_dy1, sigma_h1, n_ph1 = read_h5(file_1, ['lon', 'lat', 't_year', 'h_elv', 'dh_dx', \
                                                                              'dh_dy', 's_elv', 'n_photons'])
        
        #filter out points where lon/lat = 0
        lon1 = lon1[(lon1 !=0) & (lat1 !=0)]
        lat1 = lat1[(lon1 !=0) & (lat1 !=0)]
        #only keep points in spatial bounds
        x_min = x_bounds[0]
        x_max = x_bounds[1]
        y_min = y_bounds[0]
        y_max = y_bounds[1]
        index_1 = ((lon1 > x_min)&(lon1 < x_max) & (lat1 > y_min) & (lat1 < y_max))
        lon1 = lon1[index_1]
        if(len(lon1)) ==0:
            continue
        lat1 = lat1[index_1]
        t1 = t1[index_1]
        h1 = h1[index_1]
        dh_dx1 = dh_dx1[index_1]
        dh_dy1 = dh_dy1[index_1]
        sigma_h1 = sigma_h1[index_1]
        n_ph1 = n_ph1[index_1]
        #reproject into UTM zone 6
        lon1_p, lat1_p = transformer.transform(lat1, lon1)
        file_2 = (course_crossovers['File_2']).values[f]
        if file_1 == file_2: #skip duplicates
            continue
        #read in 2nd file
        lon2, lat2, t2, h2, dh_dx2, dh_dy2, sigma_h2, n_ph2 = read_h5(file_2, ['lon', 'lat', 't_year', 'h_elv', 'dh_dx', \
                                                                               'dh_dy', 's_elv', 'n_photons'])
        lon2 = lon2[(lon2 !=0) & (lat2 !=0)]
        lat2 = lat2[(lon2 !=0) & (lat2 !=0)]
        index_2 = ((lon2 > x_min)&(lon2 < x_max) & (lat2 > y_min) & (lat2 < y_max))
        lon2 = lon2[index_2]
        if(len(lon2)) ==0:
            continue
        lat2 = lat2[index_2]
        t2 = t2[index_2]
        h2 = h2[index_2]
        dh_dx2 = dh_dx2[index_2]
        dh_dy2 = dh_dy2[index_2]
        sigma_h2 = sigma_h2[index_2]
        n_ph2 = n_ph2[index_2]
        lon2_p, lat2_p = transformer.transform(lat2, lon2)
        #check for potential repeats (they should have all been removed in the course crossovers check, but this is to make sure)
        if((get_rgt(file_1) == get_rgt(file_2)) &(get_date(file_1) != get_date(file_2)) & \
                   (get_beam(file_1) == get_beam(file_2))):
                    distance = []
                    for i in range(0, len(lon1_p)):
                        #get distance between points on track one and closest point on track 2
                        x1 = lon1_p[i]
                        y1 = lat1_p[i]
                        distance.append(np.min(np.sqrt((x1 - lon2_p)**2 + (y1-lat2_p)**2)))
                    if np.mean(distance) < 30: #if tracks are within 30m of each other, assume it's a repeat and move to next iteration (this may be a higher threshold than needed)
                        continue
        
        #read in course crossover estimate
        lon_c = (course_crossovers['x']).values[f]
        lat_c = (course_crossovers['y']).values[f]
        #get distance from crossover location for points in each line
        distance_1 = np.sqrt((lon1_p - lon_c)**2 + (lat1_p - lat_c)**2)
        distance_2 = np.sqrt((lon2_p - lon_c)**2 + (lat2_p - lat_c)**2)
        if(radius >= 2000): #set upper bounds on initial search radius
            r2 = 10000
        else:
            r2 = 5*radius #initial search radius - start with larger radius in case our original crossover location estimate was inaccurate
        #filter for points within r2 for intermediate crossover calculation 
        lon1_f_i = lon1_p[abs(distance_1) < r2] 
        lat1_f_i = lat1_p[abs(distance_1) < r2]
        lon2_f_i = lon2_p[abs(distance_2) < r2]
        lat2_f_i = lat2_p[abs(distance_2) < r2]
        #move to next pair if there aren't any points in either line within the larger search radius
        if((len(lon1_f_i) < 2) or (len(lon2_f_i) < 2)):
            continue
        #re-calculate crossover location
        x_i, y_i = intersection(lon1_f_i, lat1_f_i, lon2_f_i, lat2_f_i)
        #make sure new crossover location is valid
        if(np.isnan(x_i)):
            continue
        #re-calculate distance from new crossover location
        distance_1_i = np.sqrt((lon1_p - x_i)**2 + (lat1_p - y_i)**2)
        distance_2_i = np.sqrt((lon2_p - x_i)**2 + (lat2_p - y_i)**2)
        #keep data for points within final search radius 
        lon1_f = lon1_p[distance_1_i < radius]
        lat1_f = lat1_p[distance_1_i < radius]
        h1_f = h1[distance_1_i < radius]
        sigma_h1_f = sigma_h1[distance_1_i < radius]
        t1_f = t1[distance_1_i < radius]
        dh_dx1_f = dh_dx1[distance_1_i < radius]
        dh_dy1_f = dh_dy1[distance_1_i < radius]
        n_ph1_f = n_ph1[distance_1_i < radius]
        
        distance_2 = np.sqrt((lon2_p - lon_c)**2 + (lat2_p - lat_c)**2)
        lon2_f = lon2_p[distance_2_i < radius]
        lat2_f = lat2_p[distance_2_i < radius]
        h2_f = h2[distance_2_i < radius]
        t2_f = t2[distance_2_i < radius]
        dh_dx2_f = dh_dx2[distance_2_i < radius]
        dh_dy2_f = dh_dy2[distance_2_i < radius]
        sigma_h2_f = sigma_h2[distance_2_i < radius]
        n_ph2_f = n_ph2[distance_2_i < radius]
        
        #make sure there are enough points before continuing
        if n_max != None:
            #sort points by distance from crossover and extract the n_max closest points
            sort_index1 = np.argsort(distance_1)
            indicies_1 = sort_index1[0:n_max]
            sort_index2 = np.argsort(distance_2)
            indicies_2 = sort_index2[0:n_max]
            lat1_f = lat1_f[indices_1]
            lon1_f = lon1_f[indices_1]
            h1_f = h1_f[indices_1]
            t1_f = t1_f[indices_1]
            sigma_h1_f = sigma_h1_f[indices_1]
            lat2_f = lat2_f[indices_2]
            lon2_f = lon2_f[indices_2]
            h2_f = h2_f[indices_2]
            t2_f = t2_f[indices_2]
            sigma_h2_f = sigma_h2_f[indices_2]
        
        if slope_filter != None: #filter by along-track slope if specified
            indices_1 = abs(dh_dx1_f) < slope_filter
            lon1_f = lon1_f[indices_1]
            lat1_f = lat1_f[indices_1]
            h1_f = h1_f[indices_1]
            sigma_h1_f = sigma_h1_f[indices_1]
            t1_f = t1_f[indices_1]
            dh_dx1_f = dh_dx1_f[indices_1]
            dh_dy1_f = dh_dy1_f[indices_1]
            n_ph1_f = n_ph1_f[indices_1]
            
            indices_2 = abs(dh_dx2_f) < slope_filter
            lon2_f = lon2_f[indices_2]
            lat2_f = lat2_f[indices_2]
            h2_f = h2_f[indices_2]
            sigma_h2_f = sigma_h2_f[indices_2]
            t2_f = t2_f[indices_2]
            dh_dx2_f = dh_dx2_f[indices_2]
            dh_dy2_f = dh_dy2_f[indices_2]
            n_ph2_f = n_ph2_f[indices_2]
        
        if (len(lon1_f) < n_min or (len(lon2_f) < n_min)): #make sure there are enough points post-filtering
            continue
            
        #re-interpolate with subsetted points to get final crossing location
        x_c_f, y_c_f = intersection(lon1_f, lat1_f, lon2_f, lat2_f)
        
        if(np.isnan(x_c_f)):
            continue
        #get along-track distance
        distance_1_f = np.sqrt((lon1_f - lon1_f[0])**2 + (lat1_f - lat1_f[0])**2)
        distance_2_f = np.sqrt((lon2_f - lon2_f[0])**2 + (lat2_f - lat2_f[0])**2)

        #get weights from uncertainties
        weights_1 = sigma_h1_f**(-1)
        #get linear fit of height as a function of along-track distance, weighted by the uncertainties on each height measurement
        p1, resid, rank, singular, rcond = np.polyfit(distance_1_f, h1_f, 1, full = True, w = weights_1, cov = 'unscaled')
        res_1 = np.polyval(p1, distance_1_f) - h1_f #residuals
        residuals_1 = (np.sum(res_1**2))#sum of the square of the residuals
        #initialize uncertainties
        sigma_m_1 = float('nan') #uncertainty on the slope
        sigma_c_1 = float('nan') #uncertainty on the intercept
        sigma_r_1 = float('nan') #uncertainty contribution of the residuals
        
        
        if(len(distance_1_f)) == 2: 
            #manually solve for slope/intercept uncertainties for 2 points
            sigma_m_1 = np.sqrt(sigma_h1_f[0]**2 + sigma_h1_f[1]**2)/(distance_1_f[1])
            sigma_r_1 = 0
            sigma_c_1 = sigma_h1_f[0]

        else:
            #get the slope/intercept uncertainties from the covariance matrix
            p1, v1 = np.polyfit(distance_1_f, h1_f, 1, w = weights_1, cov = 'unscaled')
            sigma_m_1 = np.sqrt(v1[0][0])
            sigma_c_1 = np.sqrt(v1[1][1])
            sigma_r_1 = np.sqrt(residuals_1/(len(distance_1_f) - 2))


        #get along-track distance of crossing point
        distance_c1 = np.sqrt((x_c_f - lon1_f[0])**2 + (y_c_f - lat1_f[0])**2)
        #interpolate height to crossing point
        h1_interp = np.polyval(p1, distance_c1) #fit at crossing location
        #fit and interpolate other variables:
        #time
        p1_t = np.polyfit(distance_1_f, t1_f, 1)
        t1_interp = np.polyval(p1_t, distance_c1)
        #along-track slope
        p1_dh_dx = np.polyfit(distance_1_f, dh_dx1_f, 1)
        dh_dx_i_1 = np.polyval(p1_dh_dx,distance_c1 )
        #across track slope
        p1_dh_dy = np.polyfit(distance_1_f, dh_dy1_f, 1)
        dh_dy_i_1 = np.polyval(p1_dh_dx, distance_c1)
        
        
        #repeat for second track
        weights_2 = sigma_h2_f**(-1)
        #get linear fit and the sum of the residual
        p2, resid, rank, singular, rcond = np.polyfit(distance_2_f, h2_f, 1, full = True, w = weights_2, cov = 'unscaled')
        res_2 = p2[0]*distance_2_f + p2[1] - h2_f #residuals
        residuals_2 = (np.sum(res_2**2))#sum of the square of the residuals

        if(len(distance_2_f)) == 2: 
            #manually solve for slope/intercept uncertainties for 2 points
            sigma_m_2 = np.sqrt(sigma_h2_f[0]**2 + sigma_h2_f[1]**2)/(distance_2_f[1])
            sigma_r_2 = 0
            sigma_c_2 = sigma_h2_f[0]
        else:
            #get the slope/intercept uncertainties from the covariance matrix
            p2, v2 = np.polyfit(distance_2_f, h2_f, 1, w = weights_2, cov = 'unscaled')
            sigma_m_2 = np.sqrt(v2[0][0])
            sigma_c_2 = np.sqrt(v2[1][1])
            sigma_r_2 = np.sqrt(residuals_2/(len(distance_2_f)-2))
        #get along-track distance of crossing
        distance_c2 = np.sqrt((x_c_f - lon2_f[0])**2 + (y_c_f - lat2_f[0])**2)
        h2_interp = np.polyval(p2, distance_c2)
        #interpolate other variables
        p2_t = np.polyfit(distance_2_f, t2_f, 1)
        t2_interp = np.polyval(p2_t, distance_c2)
        p2_dh_dx = np.polyfit(distance_2_f, dh_dx2_f, 1)
        dh_dx_i_2 = np.polyval(p2_dh_dx,distance_c2)
        p2_dh_dy  = np.polyfit(distance_2_f, dh_dy2_f, 1)
        dh_dy_i_2 = np.polyval(p2_dh_dy, distance_c2 )
        
        #get dh and dt at crossover location
        dh = h2_interp - h1_interp
        dt = t2_interp - t1_interp
        
        #calculate uncertainties on interpolated heights
        var_h1 = ((distance_c1 - np.mean(distance_1_f))**2)*sigma_m_1**2 +  sigma_r_1**2#variance of h1_interp
        var_h2 = ((distance_c2 - np.mean(distance_2_f))**2)*sigma_m_2**2 +  sigma_r_2**2
        sigma_dh = np.sqrt(var_h1 + var_h2) #uncertainty on crossover height difference

        #aggregate data
        crossover_x.append(x_c_f)
        crossover_y.append(y_c_f)
        height_1.append(h1_interp)
        height_2.append(h2_interp)
        dh_elev.append(dh)
        time_1.append(t1_interp)
        time_2.append(t2_interp)
        delta_time.append(dt)
        n1.append(len(lon1_f))
        n2.append(len(lon2_f))
        f1.append(file_1)
        f2.append(file_2)
        dh_dx_1.append(dh_dx_i_1)
        dh_dx_2.append(dh_dx_i_2)
        dh_dy_1.append(dh_dy_i_1)
        dh_dy_2.append(dh_dy_i_2)
        sigma_1.append(np.sqrt(var_h1))
        sigma_2.append(np.sqrt(var_h2))
        sigma_c.append(sigma_dh)
        n_photons_1.append(np.mean(n_ph1_f))
        n_photons_2.append(np.mean(n_ph2_f))
        resid_1.append(residuals_1)
        resid_2.append(residuals_2)

        
       #save data into dataframe 
    os.chdir('..')
    dt_days = [d* 365 for d in delta_time] #convert time interval from years to days
    crossover_lat, crossover_lon = transformer_i.transform(crossover_x, crossover_y) #project back into lon/lat
    #store data as dictionary
    data_dict = {'x': crossover_x, 'y': crossover_y, 'lon': crossover_lon, 'lat': crossover_lat, 'dh': dh_elev,\
                 'sigma_dh': sigma_c, 'h1_interp': height_1, 'h2_interp': height_2,'time_1': time_1, 'time_2': time_2, \
                 'dt(decimal year)': delta_time, 'dt (days)':dt_days, 'source_1': f1, 'source_2': f2, 'n_points(track_1)': n1,\
                 'n_points(track_2)': n2,'sigma_h1':sigma_1, 'sigma_h2': sigma_2, 'dh_dx_1': dh_dx_1, 'dh_dy_1': dh_dy_1, \
                'dh_dx_2': dh_dx_2, 'dh_dy_2': dh_dy_2, 'track_1_fit': resid_1, 'track_2_fit': resid_2, \
                 'n_photons_1': n_photons_1, 'n_photons_2': n_photons_2}
    
    
    data = pd.DataFrame(data_dict) #convert dictionary to dataframe
    data.to_csv(outfile) #save as csv
    end = time.time()
    print('time_elapsed: ', (end - start)) #record total time

The following cell reproduces the datasets used in Michaelides et al (2021). We tested this algorithm with interpolation radii varying from 20 to 100 m, with n_min set such that the along-track point desnity was at one segment every 40 m on average. 

In [ ]:
crossovers('course_xovers_all_slopes_new_demo.h5','Anaktuvuk_v2_ATL06_reduced_all_slopes', 20, 2,[-151.4259, -148.56], [68.5275, 69.5649],'xovers_full_period_low_slopes_new_demo.csv',slope_filter = 0.05)
#crossovers('course_xovers_all_slopes_new_demo.h5','Anaktuvuk_v2_ATL06_reduced_all_slopes', 40, 2,[-151.4259, -148.56], [68.5275, 69.5649],'xovers_40_full_period_low_slopes_new_demo.csv',slope_filter = 0.05)
#crossovers('course_xovers_all_slopes_new_demo.h5','Anaktuvuk_v2_ATL06_reduced_all_slopes', 60, 3,[-151.4259, -148.56], [68.5275, 69.5649],'xovers_60_full_period_low_slopes_new_demo.csv',slope_filter = 0.05)
#crossovers('course_xovers_all_slopes_new_demo.h5','Anaktuvuk_v2_ATL06_reduced_all_slopes', 80, 4,[-151.4259, -148.56], [68.5275, 69.5649],'xovers_80_full_period_low_slopes_new_demo.csv',slope_filter = 0.05)
#crossovers('course_xovers_all_slopes_new_demo.h5','Anaktuvuk_v2_ATL06_reduced_all_slopes', 100, 5,[-151.4259, -148.56], [68.5275, 69.5649],'xovers_100_full_period_low_slopes_new_demo.csv',slope_filter = 0.05)

# Step 3: Temporal Subsetting

In [ ]:
#pull out crossovers from individual thaw seasons (defined here as March 8th-October 22 in 2019, equivlanet DOY in 2020) 
xovers_full_new = pd.read_csv('xovers_full_period_low_slopes_new_demo.csv')
xovers_seasonal = xovers_full_new[((xovers_full_new['time_1'] - np.floor(xovers_full_new['time_1']))*365 >= 67) & \
                              ((xovers_full_new['time_2'] - np.floor(xovers_full_new['time_2']))*365 <= 295) & \
                              (np.floor(xovers_full_new['time_1']) > 2018)  &  (np.floor(xovers_full_new['time_2']) > 2018) & \
                             (np.floor(xovers_full_new['time_1']) == np.floor(xovers_full_new['time_2']))]
xovers_seasonal.reset_index(inplace = True)


In [ ]:
xovers_seasonal.to_csv('xovers_two_seasons_low_slope_new_demo.csv') #save data